In [ ]:
import pandas as pd
import xarray as xr
import numpy as np
#create dummy netcdf data
ds1 = xr.Dataset(
    {"foo": (("x", "y", "t"), np.random.rand(4, 5, 2),{'long_name':'foobar'})},
    coords={
        "x": [10, 20, 30, 40],
        "y": [1, 2, 3, 4, 5],
        "t": pd.date_range("2001-01-01", periods=2),
    }
)
ds1.to_netcdf("F:/tem/test_with_attr.nc",encoding={'foo': {'dtype': 'int8', 'add_offset': 3,'scale_factor': 0.1, '_FillValue': -9999}})
ds2 = xr.Dataset(
    {"foo": (("x", "y", "t"), np.random.rand(4, 5, 2),{'longname':'foobar'})},
    coords={
        "x": [10, 20, 30, 40],
        "y": [1, 2, 3, 4, 5],
        "t": pd.date_range("2001-01-03", periods=2),
    },
)
ds2.to_netcdf("F:/tem/test_with_attr_more.nc",encoding={'foo': {'dtype': 'int8', 'add_offset': 3,'scale_factor': 0.1, '_FillValue': -9999}})

#read in first part of data & create zarr store
ds1=xr.open_dataset('F:/tem/test_with_attr.nc',decode_cf=False,mask_and_scale=False)
ds1.close()
ds1.to_zarr('F:/tem/test_with_attr3.zarr',consolidated=True)

#read in data to append & append to zarr store
ds2=xr.open_dataset('F:/tem/test_with_attr_more.nc',decode_cf=False,mask_and_scale=False)
ds2.close()
ds2.to_zarr('F:/tem/test_with_attr.zarr',consolidated=True,mode='a',append_dim='t',)

# you will get error:
#ValueError: failed to prevent overwriting existing key add_offset in attrs. 
#This is probably an encoding field used by xarray to describe how a variable is serialized. 
#To proceed, remove this key from the variable's attributes manually.

#comment out the ds2.to_zarr above & run again

#If you followed this advice you could do this:
for var in ds2:
    tem = ds2[var].attrs
    tem.pop('add_offset')
    tem.pop('scale_factor')
    tem.pop('_FillValue')
    ds2[var].attrs=tem
ds2.to_zarr('F:/tem/test_with_attr.zarr',consolidated=True,mode='a',append_dim='t',)

#BUT this is a problem, if you read in the data, now the appended data isn't scaled right
ds2 = xr.open_zarr('F:/tem/test_with_attr.zarr',consolidated=True)
ds2

# the correct way to do this:
ds2=xr.open_dataset('F:/tem/test_with_attr_more.nc',decode_cf=False,mask_and_scale=False)
ds2.close()
ds2 = xr.decode_cf(ds2)
ds2.to_zarr('F:/tem/test_with_attr.zarr',consolidated=True,mode='a',append_dim='t',)

